In [51]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.strategies import DDPStrategy
from pytorch_lightning.loggers import CSVLogger
from transformers import BertConfig, BertForMaskedLM

In [52]:
logger = CSVLogger("mlm_logs", name='my_exp')

In [53]:
bert_config = BertConfig(vocab_size=30500)

class MLMBert(BertForMaskedLM):
    def __init__(self, config):
        super().__init__(config)
    
    def training_step(self, batch):
        # initialize calculated gradients (from prev step)
        #optim.zero_grad()

        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # process
        outputs = self.forward(input_ids, attention_mask=attention_mask,
                        labels=labels)
        print(outputs)
        # extract loss
        loss = outputs.loss

        return outputs
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=5e-5)
        return optimizer

In [54]:
model = MLMBert(config=bert_config)

In [55]:
model.configure_optimizers()

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 5e-05
    weight_decay: 0
)

In [56]:
dataset = MNIST(os.getcwd(), download=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=12)

In [57]:
len(train_loader)

1875

In [62]:

# most basic trainer, uses good defaults (auto-tensorboard, checkpoints, logs, and more)
trainer = pl.Trainer(accelerator="gpu", devices=2) 
trainer = pl.Trainer(logger=logger)

MisconfigurationException: `Trainer(strategy='ddp_spawn')` or `Trainer(accelerator='ddp_spawn')` is not compatible with an interactive environment. Run your code as a script, or choose one of the compatible strategies: Trainer(strategy=None|dp|tpu_spawn). In case you are spawning processes yourself, make sure to include the Trainer creation inside the worker function.

In [50]:
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

NameError: name 'trainer' is not defined

In [64]:
bert_config = BertConfig(vocab_size=30500)


In [65]:
class MLMBert(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.bert = BertForMaskedLM(bert_config)
        
    def training_step(self, batch):
        # initialize calculated gradients (from prev step)
        #optim.zero_grad()

        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # process
        outputs = self.forward(input_ids, attention_mask=attention_mask,
                        labels=labels)
        
        # extract loss
        loss = outputs.loss
        logger.info(loss)

        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=5e-5)
        return optimizer

In [71]:
model = MLMBert()

In [79]:
model.bert()

ValueError: You have to specify either input_ids or inputs_embeds